In [2]:
#import data to data frame and show data 
import pandas as pd
Student= pd.read_csv(r"C:\Users\Wenlei\Documents\text_files\student.txt", sep="\t")  # please change directory and sep accordingly 
Student.head()

,Studentid,StudentName
0,1,John
1,2,Mary
2,3,Adam
3,4,Terry


## DML

In [3]:
#DML operation
#select * from student
Student

,Studentid,StudentName
0,1,John
1,2,Mary
2,3,Adam
3,4,Terry


In [4]:
#select top 2 * from student
Student[:2]  #Student.head(2)

,Studentid,StudentName
0,1,John
1,2,Mary


In [5]:
#select * from student where studentid =3
Student.query('Studentid == 3')  #intuitive for database 


,Studentid,StudentName
2,3,Adam


In [6]:
#multiple conditions select * from student where studentid >2 and Studentname = 'Adam'
Student.query('Studentid > 2 and StudentName == "Adam"')

,Studentid,StudentName
2,3,Adam


In [7]:
#OR logic operation  select * from student where studentid >2 or Studentname = 'Adam'
Student.query('Studentid > 2 or StudentName == "Adam"')


,Studentid,StudentName
2,3,Adam
3,4,Terry


In [8]:
#not logic operation select * from student where studentid >2 and not( Studentname = 'Adam')
Student.query('Studentid > 2 and not(StudentName == "Adam")')

,Studentid,StudentName
3,4,Terry


In [9]:
#wild card  select * from student where studentname like '%a%'   find student name contain a
Student.query('StudentName.str.contains("a")', engine='python')


,Studentid,StudentName
1,2,Mary
2,3,Adam


In [10]:
#wild card  select * from student where studentname like 'a%'   find student name start with a
Student.query('StudentName.str.lower().str.startswith("a")', engine='python')  #notice here I use lower() function to ignore case

,Studentid,StudentName
2,3,Adam


In [11]:
# condition in a list  select * from student where studentname in ('Mary', 'Terry')
# https://stackoverflow.com/questions/19960077/how-to-filter-pandas-dataframe-using-in-and-not-in-like-in-sql
Student.query ('StudentName in (["Mary", "Terry"])')


,Studentid,StudentName
1,2,Mary
3,4,Terry


In [12]:
# condition not in a list  select * from student where studentname not in ('Mary', 'Terry')
# https://stackoverflow.com/questions/33990955/combine-pandas-dataframe-query-method-with-isin
Student.query ('StudentName not in (["Mary", "Terry"])')

,Studentid,StudentName
0,1,John
2,3,Adam


#null selection,  select * from student where studentname is not null
https://stackoverflow.com/questions/37863660/pandas-dataframe-query-fetch-not-null-rows-pandas-equivalent-to-sql-is-no


In [13]:
#select only part of columns  select studentName from student
# the following post has a good summary on use .loc and i.loc
###https://medium.com/dunder-data/selecting-subsets-of-data-in-pandas-6fcd0170be9c

Student[['StudentName']]      #it has to be a list, imaging you have more than 2 column 


,StudentName
0,John
1,Mary
2,Adam
3,Terry


In [14]:
#project a new column, select *, cast(studentid as varchar(3)) + '_' + StudentName as StudentName2 from student
#https://stackoverflow.com/questions/34283234/concatenate-index-and-string-to-new-column
Student['StudentName2'] = Student['Studentid'].astype(str) + "_" + Student['StudentName']
Student


,Studentid,StudentName,StudentName2
0,1,John,1_John
1,2,Mary,2_Mary
2,3,Adam,3_Adam
3,4,Terry,4_Terry


In [15]:
#drop a column
Student= Student.drop(['StudentName2'], axis=1)   #replace the old dataframe
Student


,Studentid,StudentName
0,1,John
1,2,Mary
2,3,Adam
3,4,Terry


In [16]:
#insert data  https://pythonexamples.org/pandas-dataframe-add-append-row/
# insert into student values (5, 'George')
#create a dictionary
new_row = {'Studentid': 5, 'StudentName':'George'}
Student = Student.append(new_row, ignore_index=True)
Student

,Studentid,StudentName
0,1,John
1,2,Mary
2,3,Adam
3,4,Terry
4,5,George


In [17]:
#update a row   # update student set StudentName = 'Lisa' where studentid =5
#https://stackoverflow.com/questions/24036911/how-to-update-values-in-a-specific-row-in-a-python-pandas-dataframe
#Student.loc[Student.Studentid==5, Student.StudentName] = 'Lisa'
#Student
Student.loc[Student.Studentid==5, 'StudentName'] ='Lisa'   #use row and column index to locate George and replace the value
Student

,Studentid,StudentName
0,1,John
1,2,Mary
2,3,Adam
3,4,Terry
4,5,Lisa


In [18]:
#delete a row   delete from student where studentid =5
Student = Student.loc[Student.Studentid!=5]
Student


,Studentid,StudentName
0,1,John
1,2,Mary
2,3,Adam
3,4,Terry


In [19]:
#order by   select * from student order by studentname
Student.sort_values('StudentName')

,Studentid,StudentName
2,3,Adam
0,1,John
1,2,Mary
3,4,Terry


In [20]:
#order by descend  select * from student order by studentname desc
Student.sort_values('StudentName', ascending=False)

,Studentid,StudentName
3,4,Terry
1,2,Mary
0,1,John
2,3,Adam


#select distinct column or row
https://riptutorial.com/pandas/example/26077/select-distinct-rows-across-dataframe

In [21]:
#join two dataset 
#import additional table 1
Course= pd.read_csv(r"C:\Users\Wenlei\Documents\text_files\course.txt", sep="\t")
Course

,Courseid,Course,Tuition
0,a,English,200
1,b,Big Data,600
2,c,Business,400
3,d,Biology,300


In [22]:
#becasue course table is many to many relation with student table, so need a bridge table
StudentCourse= pd.read_csv(r"C:\Users\Wenlei\Documents\text_files\studentcourse.txt", sep="\t")
StudentCourse

,Studentid,Courseid
0,1,a
1,1,b
2,2,d
3,3,a
4,3,c
5,4,b
6,4,c


In [23]:
#let us see how we join three table together to get some useful info
#use inner join        select * from student s inner join studentcourse sc on s.studentid = sc.studentid inner join course c on sc.courseid = c.courseid 
fulldata1 = pd.merge(Student, StudentCourse, how='inner', on='Studentid')  #first join student with studentcourse
fulldata = pd.merge(fulldata1, Course, how='inner', on='Courseid')
fulldata

,Studentid,StudentName,Courseid,Course,Tuition
0,1,John,a,English,200
1,3,Adam,a,English,200
2,1,John,b,Big Data,600
3,4,Terry,b,Big Data,600
4,2,Mary,d,Biology,300
5,3,Adam,c,Business,400
6,4,Terry,c,Business,400


In [24]:
# to see left join effect, i will manually add a student to student table 
new_row = {'Studentid': 5, 'StudentName':'George'}
Student = Student.append(new_row, ignore_index=True)
print (Student)
#left join to see effect, George has no value on the course side 
fulldata1 = pd.merge(Student, StudentCourse, how='left', on='Studentid')  #first join student with studentcourse
fulldata = pd.merge(fulldata1, Course, how='left', on='Courseid')
fulldata

   Studentid StudentName
0          1        John
1          2        Mary
2          3        Adam
3          4       Terry
4          5      George


,Studentid,StudentName,Courseid,Course,Tuition
0,1,John,a,English,200.0
1,1,John,b,Big Data,600.0
2,2,Mary,d,Biology,300.0
3,3,Adam,a,English,200.0
4,3,Adam,c,Business,400.0
5,4,Terry,b,Big Data,600.0
6,4,Terry,c,Business,400.0
7,5,George,NaN,NaN,NaN


In [25]:
#check set operation, manually create a dataframe call Student2
import numpy as np
Student2= pd.DataFrame(np.array([
    #[1, 'John'],
    Student.iloc[0],   #replicate this from Student
    [6, 'Katie'],
    [7, 'Joe']]),
    columns=['Studentid', 'StudentName'])
Student2

,Studentid,StudentName
0,1,John
1,6,Katie
2,7,Joe


In [26]:
#test union and union all with the newly create dataset, I intentionally include john and different name
#select * from student union all select * from student2
Student_unionall = pd.concat([Student, Student2],ignore_index=True)  #ignore_index will reset 2nd dataframe index 
Student_unionall
#you can see John appear twice

,Studentid,StudentName
0,1,John
1,2,Mary
2,3,Adam
3,4,Terry
4,5,George
5,1,John
6,6,Katie
7,7,Joe


In [27]:
# test union  select * from student union select * from student2
Student_union = pd.concat([Student, Student2],ignore_index=True).drop_duplicates()  #ignore_index will reset 2nd dataframe index 
Student_union

,Studentid,StudentName
0,1,John
1,2,Mary
2,3,Adam
3,4,Terry
4,5,George
6,6,Katie
7,7,Joe


In [28]:
#test intersect    select * from student intersect select * from student2
Student_insersect = pd.merge(Student, Student2)
Student_insersect

,Studentid,StudentName
0,1,John


In [29]:
#test except or minus   setdiff = original dataset - intersect 
# select * from student except select * from student2
#https://stackoverflow.com/questions/33990955/combine-pandas-dataframe-query-method-with-isin
except_list = list(Student_insersect.StudentName)
Student_except =Student.query ('StudentName != @except_list')
Student_except
#except_list

,Studentid,StudentName
1,2,Mary
2,3,Adam
3,4,Terry
4,5,George


## Analytics 

In [30]:
#show data we worked with
fulldata

,Studentid,StudentName,Courseid,Course,Tuition
0,1,John,a,English,200.0
1,1,John,b,Big Data,600.0
2,2,Mary,d,Biology,300.0
3,3,Adam,a,English,200.0
4,3,Adam,c,Business,400.0
5,4,Terry,b,Big Data,600.0
6,4,Terry,c,Business,400.0
7,5,George,NaN,NaN,NaN


In [38]:
#get sum of tuition for each student   select studentName, sum(tuition) from fulldata group by studentName
#https://stackoverflow.com/questions/53781634/aggregation-in-pandas
#https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.08-Aggregation-and-Grouping.ipynb

fulldata.groupby('StudentName' ,as_index=False).agg({'Tuition': 'sum'})   #as_index = False don't use parent index

,StudentName,Tuition
0,Adam,600.0
1,George,0.0
2,John,800.0
3,Mary,300.0
4,Terry,1000.0


In [43]:
#get sum of tuition for each student   select studentName, sum(tuition) as tuitiontotal, count(*) as coursecount from fulldata group by studentName
#need to resetindex to rename
fulldata.groupby('StudentName').agg({'Tuition':'sum', 'Courseid': 'count'}).reset_index().rename(columns={'Tuition':'TuitionTotal', 'Courseid':'CourseCount'})


,StudentName,TuitionTotal,CourseCount
0,Adam,600.0,2
1,George,0.0,0
2,John,800.0,2
3,Mary,300.0,1
4,Terry,1000.0,2


In [37]:
#having clasue    select studentName, sum(tuition) from fulldata group by studentname having sum(tuition) > 600
#http://www.notespoint.com/pandas-groupby/
datasum= fulldata.groupby('StudentName',as_index=False).agg({'Tuition': 'sum'})
datasum[datasum['Tuition'] >600]

,StudentName,Tuition
2,John,800.0
4,Terry,1000.0
